In [3]:
import pandas as pd

# Şikayetlerin varyasyonları ve genişletilmiş liste
specialty_complaints =  {
    "Nöroloji": ["baş ağrısı", "migren", "baş dönmesi", "uykusuzluk", "zihinsel yorgunluk", "nöbet", "beyin", "epilepsi", "alzheimer", "parkison"],
    "Cildiye": ["cilt döküntüsü", "kaşıntı", "deri", "akne", "sivilce", "güneş lekesi", "alerji", "egzama", "dermatit", "sedef"],
    "Göz Hastalıkları": ["bulanık görme", "göz ağrısı", "göz kuruluğu", "görme kaybı", "göz enfeksiyonu", "katarakt", "glokom", "göz yanması", "gözlük", "göz kayması"],
    "Dahiliye": ["kan şekeri", "hipertansiyon", "kolesterol", "diyabet", "kalp hastalıkları", "astım", "obezite", "tiroit", "yüksek tansiyon", "soğuk algınlığı"],
    "Genel Cerrahi": ["karın ağrısı", "apandisit", "sindirim problemleri", "karaciğer hastalıkları", "kolesistit", "gastrit", "safra kesesi", "şişmanlık", "mide bulantısı", "kilo kaybı"],
    "Kardiyoloji": ["kalp ağrısı", "hipertansiyon", "koroner arter hastalığı", "kalp krizi", "kolesterol", "kalp ritmi", "tansiyon", "nefes darlığı", "yüksek kolesterol", "bypass"],
    "Kulak Burun Boğaz": ["burun tıkanıklığı", "sinüs enfeksiyonu", "boğaz ağrısı", "kulak ağrısı", "işitme kaybı", "zorlukla nefes almak", "ses kısıklığı", "öksürük", "sindirimi güçlük", "burun akıntısı"],
    "Jinekoloji": ["adet düzensizliği", "gebelik", "doğum kontrolü", "kısırlık", "menopoz", "vajinal enfeksiyon", "rahim hastalıkları", "rahim kanseri", "karnı ağrısı", "çocuk doğurma"],
    "Pediatri": ["çocuk sağlığı", "büyüme bozuklukları", "ateş", "cilt döküntüsü", "kulak ağrısı", "aşı", "grip", "zihinsel gelişim", "çocuk gelişimi", "gelişimsel gerilik"],
    "Üroloji": ["idrar yolu enfeksiyonu", "sistit", "prostat", "böbrek taşı", "idrar kaçırma", "testis ağrısı", "idrar retansiyonu", "erektil disfonksiyon", "idrar yapma zorluğu", "mesane"],
    "Ortopedi": ["sırt ağrısı", "eklem ağrısı", "kas ağrısı", "kemik kırığı", "artrit", "kireçlenme", "bacak ağrısı", "boyun ağrısı", "omuz ağrısı", "fiziksel tedavi"],
    "Psikiyatri": ["depresyon", "anksiyete", "panik atak", "stres", "kaygı", "zihinsel sağlık", "intihar", "psikolojik destek", "bipolar", "sosyal fobi"]
}

# Veri setini oluşturma
data = []

# Her şikayet ve ona karşılık gelen uzmanlıkları eşleştir
for specialty, complaints_list in specialty_complaints.items():
    for complaint in complaints_list:
        data.append({"Complaint": complaint, "Specialty": specialty})

df = pd.DataFrame(data)

# Veri setini CSV dosyasına kaydetme
df.to_csv('complaints_speciality.csv', index=False)

# Veri setini inceleme
print(df.head())



            Complaint Specialty
0          baş ağrısı  Nöroloji
1              migren  Nöroloji
2         baş dönmesi  Nöroloji
3          uykusuzluk  Nöroloji
4  zihinsel yorgunluk  Nöroloji


In [3]:
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import pandas as pd

# Cihaz seçimi (GPU varsa kullan, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Veri setini yükleme
df = pd.read_csv('complaints_speciality.csv')

# Şikayet metinlerini ve uzmanlıkları ayıralım
texts = df['Complaint'].tolist()
labels = df['Specialty'].tolist()

# Etiketlerin sayısal hale getirilmesi
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Eğitim ve test setlerine ayıralım
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# BERT tokenizer'ı yükleyelim
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize etme fonksiyonu
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

# Veriyi tokenleştirerek uygun formata dönüştürme
train_encodings = tokenize_function(train_texts)
val_encodings = tokenize_function(val_texts)

# Dataset objesi oluşturma
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

# Modeli yükleyelim
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Modeli doğru cihaza taşıyalım
model.to(device)

# Eğitim parametrelerini ayarlama
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=200,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

# Trainer objesini oluşturma
trainer = Trainer(
    model=model,                         # the model to be trained
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

# Modeli eğitme
trainer.train()

# Modeli kaydetme
model.save_pretrained('bert_specialty_model')

# Kullanıcıdan şikayet alma ve tahmin yapma
def predict_specialty(complaint):
    encoding = tokenizer(complaint, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Modeli değerlendirme modunda kullanarak tahmin yapma
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # En yüksek olasılığa sahip etiketi almak
    predicted_label = torch.argmax(logits, dim=1).item()

    # Etiketin uzmanlık karşılığını almak
    predicted_specialty = label_encoder.inverse_transform([predicted_label])[0]
    return predicted_specialty

# Test: Kullanıcıdan şikayet almak ve tahmin yapmak
user_complaint = "Sürekli yorgunum"  # Örnek şikayet
recommended_specialty = predict_specialty(user_complaint)
print(f"Önerilen uzmanlık: {recommended_specialty}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,1.209300
1000,0.043100
1500,0.036900
2000,0.033300


Önerilen uzmanlık: Kulak Burun Boğaz


Resetleme kodu

In [6]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Eğitimde kullanılan veriyi tekrar oku (etiket sırasını aynı tutmak için)
df = pd.read_csv('complaints_speciality.csv')
labels = df['Specialty'].tolist()

# Etiketleri tekrar kodla (aynı sıralamayla)
label_encoder = LabelEncoder()
label_encoder.fit(labels)

# Tokenizer ve eğitilmiş modeli yükle
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert_specialty_model')

# Cihaza taşı
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_specialty(complaint):
    encoding = tokenizer(complaint, return_tensors='pt', padding=True, truncation=True, max_length=128)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    predicted_label = torch.argmax(logits, dim=1).item()
    predicted_specialty = label_encoder.inverse_transform([predicted_label])[0]
    return predicted_specialty


PKL dosyasına label encoder'i kaydetme

In [8]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

# Veriyi yükle
df = pd.read_csv('complaints_speciality.csv')
labels = df['Specialty'].tolist()

# LabelEncoder oluştur
label_encoder = LabelEncoder()
label_encoder.fit(labels)

# Kaydet
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


In [7]:
user_complaint = "kalp"  # Örnek şikayet
recommended_specialty = predict_specialty(user_complaint)
print(f"Önerilen uzmanlık: {recommended_specialty}")

Önerilen uzmanlık: Kardiyoloji
